In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline
os.chdir('Ultrasounds')
if os.path.isdir('train/Abnormal') is False:
    os.makedirs('train/Abnormal')
    os.makedirs('train/Normal')
    os.makedirs('valid/Abnormal')
    os.makedirs('valid/Normal')
    os.makedirs('test/Abnormal')
    os.makedirs('test/Normal')
    for i in random.sample(glob.glob('Normal*'), 75):
        shutil.move(i, 'train/Normal')      
    for i in random.sample(glob.glob('Abnormal*'), 200):
        shutil.move(i, 'train/Abnormal')
    for i in random.sample(glob.glob('Normal*'), 20):
        shutil.move(i, 'valid/Normal')        
    for i in random.sample(glob.glob('Abnormal*'), 100):
        shutil.move(i, 'valid/Abnormal')
    for i in random.sample(glob.glob('Normal*'), 5):
        shutil.move(i, 'test/Normal')      
    for i in random.sample(glob.glob('Abnormal*'), 50):
        shutil.move(i, 'test/Abnormal')
os.chdir('../../')
train_path = 'Ultrasounds/train'
valid_path = 'Ultrasounds/valid'
test_path = 'Ultrasounds/test'
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(224,224), classes=['Abnormal', 'Normal'], batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(224,224), classes=['Abnormal', 'Normal'], batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(224,224), classes=['Abnormal', 'Normal'], batch_size=5, shuffle=False)
    imgs, labels = next(train_batches)
    def plotImages(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()
plotImages(imgs)
print(labels)
model = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding = 'same', input_shape=(224,224,3)),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding = 'same'),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Flatten(),
    Dense(units=2, activation='softmax')
])
model.summary()
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x=train_batches,
    steps_per_epoch=len(train_batches),
    validation_data=valid_batches,
    validation_steps=len(valid_batches),
    epochs=5,
    verbose=2
)
model.save("Usmodel.h5")
predictions = model.predict(x=test_batches, steps=len(test_batches), verbose=0)
np.round(predictions)
cm = confusion_matrix(y_true=test_batches.classes, y_pred=np.argmax(predictions, axis=-1))
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    cm_plot_labels = ['Normal','Abnormal']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')